In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import pickle

import seaborn as sns

%matplotlib qt
mpl.rcParams['lines.linewidth'] = 0.91
plt.style.use('seaborn-v0_8-whitegrid')

In [2]:
from functions.bursts import hl_envelopes_idx, detect_bursts, merge_excluding, is_isolated, find_isolated_combination, find_combined_movements_all_limbs
from functions.acc_utils import compute_acc_norm
from functions.posture import compute_spherical_coordinates, detect_posture_changes

In [23]:
subjects = ["158", "098", "633", "906", "279", "547", "971", "958", "815", "127", "914", "965"]
subjects = ["098"]
for i, sub in enumerate(subjects):
    print(sub)
    trunk_acc_df = pd.read_pickle(f"/Volumes/Untitled/rehab/data/{sub}/acc_night/trunk_acc_3axes.pkl")
    break

098


### xAngle, yAngle, zAngle

In [5]:
x_angle = (np.arctan(trunk_acc_df['x'].rolling('5 s').mean() /
                     np.sqrt(trunk_acc_df['y'].rolling('5 s').mean()**2 + trunk_acc_df['z'].rolling('5 s').mean()**2)) * (180 / np.pi)
                     ).resample('5 s').mean()

y_angle = (np.arctan(trunk_acc_df['y'].rolling('5 s').mean() /
                        np.sqrt(trunk_acc_df['x'].rolling('5 s').mean()**2 + trunk_acc_df['z'].rolling('5 s').mean()**2)) * (180 / np.pi)
                        ).resample('5 s').mean()

z_angle = (np.arctan(np.sqrt(trunk_acc_df['x'].rolling('5 s').mean()**2 + trunk_acc_df['y'].rolling('5 s').mean()**2) /
                        trunk_acc_df['z'].rolling('5 s').mean()) * (180 / np.pi)
                        ).resample('5 s').mean()

In [24]:
phi, theta = compute_spherical_coordinates(trunk_acc_df.resample('10s').median())
trunk_acc_sph = pd.DataFrame({"phi": phi * 180 / np.pi, "theta": theta * 180 / np.pi}, index=trunk_acc_df.resample('10s').median().index)
updated_df = detect_posture_changes(trunk_acc_sph.copy())
time_posture_change30 = updated_df[updated_df['posture_change30']].index
time_posture_change10 = updated_df[updated_df['posture_change10']].index

/Users/marcellosicbaldi/anaconda3/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/src/functions/posture.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[134.07317921579272, 176.15650121316543, 168.94881106601036, 30.220508674815672, 172.64340815394462, 33.44078552110278, 162.02076195912946, 178.06676352473266, 30.728468108588952, 146.2396052146629, 58.949917343336146, 119.04853899898889, 111.52204120658573, 89.59162298612216, 169.8986380019962, 33.87512061452872, 36.425053256876225, 32.135461077621954, 70.80394278399335, 79.8842947216979, 106.37231252422335, 164.43856756753593, 163.36527914258943, 160.9524065236717, 173.89473730391356, 145.59172047372235]' has dtype incompatible with int64, please explicitly cast to a compatibl

In [27]:
plt.figure(figsize=(10, 6))
plt.plot(trunk_acc_sph['theta'], label='theta')

for t in time_posture_change30:
    plt.axvline(t, color='r', linestyle='--', alpha=0.5)

In [25]:
trunk_acc_sph["position"] = "p"
trunk_acc_sph.loc[(trunk_acc_sph["theta"] > -45) & (trunk_acc_sph["theta"] < 45), "position"] = "supine"
trunk_acc_sph.loc[(trunk_acc_sph["theta"] > -135) & (trunk_acc_sph["theta"] < -45), "position"] = "right"
trunk_acc_sph.loc[(trunk_acc_sph["theta"] > 45) & (trunk_acc_sph["theta"] < 135), "position"] = "left"
trunk_acc_sph.loc[(trunk_acc_sph["theta"] > 135) | (trunk_acc_sph["theta"] < -135), "position"] = "prone"

trunk_acc_sph

,phi,theta,position
time,,,
2024-03-16 02:01:00,17.147765,120.975987,left
2024-03-16 02:01:10,56.004327,-104.950834,right
2024-03-16 02:01:20,95.080352,-114.974360,right
2024-03-16 02:01:30,129.980006,-90.428707,right
2024-03-16 02:01:40,135.037326,-84.456072,right
...,...,...,...
2024-03-16 09:49:10,22.134008,119.745118,left
2024-03-16 09:49:20,21.583176,121.436459,left
2024-03-16 09:49:30,21.637852,122.088199,left


In [26]:
df_PC = pd.DataFrame({"posture_change_degrees30": updated_df[updated_df['posture_change30']]["posture_change_degrees30"]}, index = pd.to_datetime(time_posture_change30))
df_PC["position"] = trunk_acc_sph.loc[time_posture_change30, "position"]
to_add = pd.DataFrame({'posture_change_degrees30': np.nan, 'position': trunk_acc_sph["position"].iloc[0]}, index = pd.DatetimeIndex([updated_df.index[0]]))
df = pd.concat([to_add, df_PC])
df_PC['previous_position'] = df_PC['position'].shift(1)
# Create a column representing the transition between postures
df_PC['transition'] = df_PC['previous_position'] + '2' + df_PC['position']
# Replace the transition for the last row, as it will be NaN after shifting
# df_PC.iloc[-1, df_PC.columns.get_loc('transition')] = np.nan
# We can drop the 'Next_Position' column as it's no longer needed
df_PC.drop('previous_position', axis=1, inplace=True)
# df_PC.drop("position", axis=1, inplace=True)
start_posture = trunk_acc_sph.loc[df_PC.index[0] - pd.Timedelta('10s')]["position"]
df_PC.loc[df_PC.index[0], "transition"] = start_posture + '2' + df_PC.loc[df_PC.index[0], "position"]
df_PC

,posture_change_degrees30,position,transition
time,,,
2024-03-16 02:01:10,134.073179,right,left2right
2024-03-16 03:07:40,176.156501,left,right2left
2024-03-16 03:51:10,168.948811,right,left2right
2024-03-16 04:03:40,30.220509,right,right2right
2024-03-16 04:47:20,172.643408,left,right2left
2024-03-16 04:47:30,33.440786,left,left2left
2024-03-16 05:03:50,162.020762,right,left2right
2024-03-16 05:10:10,178.066764,left,right2left
2024-03-16 05:18:50,30.728468,left,left2left


In [7]:
updated_df

,phi,theta,time_seconds,posture_change30,posture_change_degrees30,posture_change10,posture_change_degrees10
time,,,,,,,
2024-03-16 02:01:00,17.147765,120.975987,0.0,False,0.000000,False,0.000000
2024-03-16 02:01:10,56.004327,-104.950834,10.0,True,134.073179,True,134.073179
2024-03-16 02:01:20,95.080352,-114.974360,20.0,False,0.000000,True,10.023526
2024-03-16 02:01:30,129.980006,-90.428707,30.0,False,0.000000,True,24.545653
2024-03-16 02:01:40,135.037326,-84.456072,40.0,False,0.000000,False,0.000000
...,...,...,...,...,...,...,...
2024-03-16 09:49:10,22.134008,119.745118,28090.0,False,0.000000,False,0.000000
2024-03-16 09:49:20,21.583176,121.436459,28100.0,False,0.000000,False,0.000000
2024-03-16 09:49:30,21.637852,122.088199,28110.0,False,0.000000,False,0.000000


In [13]:
plt.figure(figsize=(15, 10))
colors = {'prone': 'skyblue', 'right': 'lightgreen', 'left': 'salmon', 'supine': 'lightcoral'}

plt.plot(trunk_acc_sph["theta"], linewidth=2.1)

# Plot the different positions
for position in trunk_acc_sph["position"].unique():
    plt.fill_between(trunk_acc_sph.index, trunk_acc_sph["theta"], where=trunk_acc_sph["position"] == position, color=colors[position], alpha=0.5)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel('Time', fontsize=19)
plt.ylabel('Azimuthal angle (deg)', fontsize=19)
plt.title('Posture Changes During Sleep', fontsize=21)

# Handling labels for the legend
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), fontsize = 21)

In [9]:
import neurokit2 as nk

In [10]:
for i, sub in enumerate(subjects):

    print(sub)

    trunk_df = pd.read_pickle(f"/Volumes/Untitled/rehab/data/{sub}/acc_night/trunk.pkl") * 1000
    ll_df = pd.read_pickle(f"/Volumes/Untitled/rehab/data/{sub}/acc_night/ll.pkl") * 1000
    rl_df = pd.read_pickle(f"/Volumes/Untitled/rehab/data/{sub}/acc_night/rl.pkl") * 1000
    lw_df = pd.read_pickle(f"/Volumes/Untitled/rehab/data/{sub}/acc_night/lw.pkl") * 1000
    rw_df = pd.read_pickle(f"/Volumes/Untitled/rehab/data/{sub}/acc_night/rw.pkl") * 1000
    trunk_acc_df = pd.read_pickle(f"/Volumes/Untitled/rehab/data/{sub}/acc_night/trunk_acc_3axes.pkl")

    lw_df_bp = pd.Series(nk.signal_filter(lw_df.values, sampling_rate = 100, lowcut=0.1, highcut=10, method='butterworth', order=8), index = lw_df.index)
    rw_df_bp = pd.Series(nk.signal_filter(rw_df.values, sampling_rate = 100, lowcut=0.1, highcut=10, method='butterworth', order=8), index = rw_df.index)
    ll_df_bp = pd.Series(nk.signal_filter(ll_df.values, sampling_rate = 100, lowcut=0.1, highcut=10, method='butterworth', order=8), index = ll_df.index)
    rl_df_bp = pd.Series(nk.signal_filter(rl_df.values, sampling_rate = 100, lowcut=0.1, highcut=10, method='butterworth', order=8), index = rl_df.index)
    trunk_df_bp = pd.Series(nk.signal_filter(trunk_df.values, sampling_rate = 100, lowcut=0.1, highcut=10, method='butterworth', order=8), index = trunk_df.index)

    bursts_lw = detect_bursts(lw_df_bp, plot = False, alfa = 20)
    bursts_rw = detect_bursts(rw_df_bp, plot = False, alfa = 20)
    bursts_ll = detect_bursts(ll_df_bp, plot = False, alfa = 15)
    bursts_rl = detect_bursts(rl_df_bp, plot = False, alfa = 15)
    bursts_trunk = detect_bursts(trunk_df_bp, plot = False, alfa = 15)

    ### POSTURE ###
    phi, theta = compute_spherical_coordinates(trunk_acc_df.resample('10s').median())
    trunk_acc_sph = pd.DataFrame({"phi": phi * 180 / np.pi, "theta": theta * 180 / np.pi}, index=trunk_acc_df.resample('10s').median().index)
    updated_df = detect_posture_changes(trunk_acc_sph.copy())
    time_posture_change30 = updated_df[updated_df['posture_change30']].index
    time_posture_change10 = updated_df[updated_df['posture_change10']].index
    trunk_acc_sph["position"] = "p"
    trunk_acc_sph.loc[(trunk_acc_sph["theta"] > -45) & (trunk_acc_sph["theta"] < 45), "position"] = "supine"
    trunk_acc_sph.loc[(trunk_acc_sph["theta"] > -135) & (trunk_acc_sph["theta"] < -45), "position"] = "right"
    trunk_acc_sph.loc[(trunk_acc_sph["theta"] > 45) & (trunk_acc_sph["theta"] < 135), "position"] = "left"
    trunk_acc_sph.loc[(trunk_acc_sph["theta"] > 135) | (trunk_acc_sph["theta"] < -135), "position"] = "prone"

    df_PC = pd.DataFrame({"posture_change_degrees30": updated_df[updated_df['posture_change30']]["posture_change_degrees30"]}, index = pd.to_datetime(time_posture_change30))
    df_PC["position"] = trunk_acc_sph.loc[time_posture_change30, "position"]
    to_add = pd.DataFrame({'posture_change_degrees30': np.nan, 'position': trunk_acc_sph["position"].iloc[0]}, index = pd.DatetimeIndex([updated_df.index[0]]))
    df = pd.concat([to_add, df_PC])
    df_PC['previous_position'] = df_PC['position'].shift(1)
    # Create a column representing the transition between postures
    df_PC['transition'] = df_PC['previous_position'] + '2' + df_PC['position']
    # We can drop the 'Next_Position' column as it's no longer needed
    df_PC.drop('previous_position', axis=1, inplace=True)

    # join bursts and posture changes
    bursts_lw["posture_change"] = 0
    bursts_rw["posture_change"] = 0
    bursts_ll["posture_change"] = 0
    bursts_rl["posture_change"] = 0
    bursts_trunk["posture_change"] = 0
    bursts_lw["transition"] = "None"
    bursts_rw["transition"] = "None"
    bursts_ll["transition"] = "None"
    bursts_rl["transition"] = "None"
    bursts_trunk["transition"] = "None"


    for time in time_posture_change30:
        for i in range(len(bursts_lw)):
            if time > bursts_lw["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_lw["end"].iloc[i]+pd.Timedelta(seconds = 5):
                bursts_lw["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees30"]
                bursts_lw["transition"].iloc[i] = df_PC.loc[time, "transition"]
        for i in range(len(bursts_rw)):
            if time > bursts_rw["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_rw["end"].iloc[i]+pd.Timedelta(seconds = 5):
                bursts_rw["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees30"]
                bursts_rw["transition"].iloc[i] = df_PC.loc[time, "transition"]
        for i in range(len(bursts_ll)):
            if time > bursts_ll["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_ll["end"].iloc[i]+pd.Timedelta(seconds = 5):
                bursts_ll["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees30"]
                bursts_ll["transition"].iloc[i] = df_PC.loc[time, "transition"]
        for i in range(len(bursts_rl)):
            if time > bursts_rl["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_rl["end"].iloc[i]+pd.Timedelta(seconds = 5):
                bursts_rl["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees30"]
                bursts_rl["transition"].iloc[i] = df_PC.loc[time, "transition"]
        for i in range(len(bursts_trunk)):
            if time > bursts_trunk["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_trunk["end"].iloc[i]+pd.Timedelta(seconds = 5):
                bursts_trunk["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees30"]
                bursts_trunk["transition"].iloc[i] = df_PC.loc[time, "transition"]

    for time in time_posture_change10:
        for i in range(len(bursts_lw)):
            if time > bursts_lw["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_lw["end"].iloc[i]+pd.Timedelta(seconds = 5):
                bursts_lw["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees10"]
        for i in range(len(bursts_rw)):
            if time > bursts_rw["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_rw["end"].iloc[i]+pd.Timedelta(seconds = 5):
                bursts_rw["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees10"]
        for i in range(len(bursts_ll)):
            if time > bursts_ll["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_ll["end"].iloc[i]+pd.Timedelta(seconds = 5):
                bursts_ll["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees10"]
        for i in range(len(bursts_rl)):
            if time > bursts_rl["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_rl["end"].iloc[i]+pd.Timedelta(seconds = 5):
                bursts_rl["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees10"]
        for i in range(len(bursts_trunk)):
            if time > bursts_trunk["start"].iloc[i]-pd.Timedelta(seconds = 5) and time < bursts_trunk["end"].iloc[i]+pd.Timedelta(seconds = 5):
                bursts_trunk["posture_change"].iloc[i] = updated_df.loc[time, "posture_change_degrees10"]
    break

098


/Users/marcellosicbaldi/anaconda3/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/src/functions/posture.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[134.07317921579272, 176.15650121316543, 168.94881106601036, 30.220508674815672, 172.64340815394462, 33.44078552110278, 162.02076195912946, 178.06676352473266, 30.728468108588952, 146.2396052146629, 58.949917343336146, 119.04853899898889, 111.52204120658573, 89.59162298612216, 169.8986380019962, 33.87512061452872, 36.425053256876225, 32.135461077621954, 70.80394278399335, 79.8842947216979, 106.37231252422335, 164.43856756753593, 163.36527914258943, 160.9524065236717, 173.89473730391356, 145.59172047372235]' has dtype incompatible with int64, please explicitly cast to a compatibl

In [11]:
bursts_lw

(                            start                           end  \
 0   2024-03-16 02:01:00.998850107 2024-03-16 02:01:55.388849974   
 1   2024-03-16 02:09:13.058850050 2024-03-16 02:09:14.138849974   
 2   2024-03-16 02:13:11.748850107 2024-03-16 02:13:12.188849926   
 3   2024-03-16 02:14:53.628849983 2024-03-16 02:14:54.268850088   
 4   2024-03-16 02:17:24.198849916 2024-03-16 02:17:25.318850040   
 ..                            ...                           ...   
 100 2024-03-16 09:32:02.838850021 2024-03-16 09:32:03.728849888   
 101 2024-03-16 09:33:41.668849945 2024-03-16 09:33:42.858850002   
 102 2024-03-16 09:39:39.508850098 2024-03-16 09:39:40.518850088   
 103 2024-03-16 09:48:27.818850040 2024-03-16 09:48:40.118849993   
 104 2024-03-16 09:49:18.348850012 2024-03-16 09:49:25.808850050   
 
                      duration  peak-to-peak           AUC  posture_change  \
 0   0 days 00:00:54.389999867   3028.857133  28215.554828       24.545653   
 1   0 days 00:00:01.07999

In [12]:
bursts_rl

,start,end,duration,peak-to-peak,AUC,posture_change,transition
0,2024-03-16 02:01:11.313223839,2024-03-16 02:01:57.953223705,0 days 00:00:46.639999866,1390.415897,11704.999547,24.545653,NaN
1,2024-03-16 02:02:49.393223763,2024-03-16 02:02:52.383223772,0 days 00:00:02.990000009,61.747458,81.319945,0.000000,t
2,2024-03-16 02:03:12.503223895,2024-03-16 02:03:13.623223782,0 days 00:00:01.119999887,36.201236,25.964204,0.000000,t
3,2024-03-16 02:07:06.883223772,2024-03-16 02:07:08.593223810,0 days 00:00:01.710000038,67.182775,75.817591,0.000000,t
4,2024-03-16 02:08:15.313223839,2024-03-16 02:08:16.623223782,0 days 00:00:01.309999943,37.286883,20.059361,0.000000,t
...,...,...,...,...,...,...,...
143,2024-03-16 09:25:31.613223791,2024-03-16 09:25:35.253223896,0 days 00:00:03.640000105,303.746073,259.744943,0.000000,t
144,2024-03-16 09:29:27.433223486,2024-03-16 09:29:42.293224096,0 days 00:00:14.860000610,730.120762,2020.716301,0.000000,t
145,2024-03-16 09:48:29.453223705,2024-03-16 09:48:41.483223677,0 days 00:00:12.029999972,271.244452,931.748768,145.591720,NaN
146,2024-03-16 09:49:16.813223839,2024-03-16 09:49:18.043224096,0 days 00:00:01.230000257,15.034092,10.538649,0.000000,t
